In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install dreamsboard -U

### 介绍
使用langchain进行任务规划，构建子任务的会话场景资源，通过MCTS任务执行器，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


## 使用

### 构建任务

- 初始化任务引擎 StructuredTaskStepStoryboard传入需要的任务

- loader_task_step_iter_builder 构建任务的子任务，完成后SimpleTaskStepStore可获取子任务信息

- init_task_engine_dreams 初始化场景加载资源，对子任务进行规划，获取会话的资源信息

```

    os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
    os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
    os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
    os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
    os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
    os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST
```

- init_task_engine_storyboard_executor 构建会话场景执行器，初始化一个会话

- storyboard_code_gen_builder 构建会话场景执行器, 对会话存储进行加载，加载失败重新构建

- generate_step_answer 通过会话场景 获取任务的答案





```

In [3]:
   
from dreamsboard.engine.storage.task_step_store.types import BaseTaskStepStore
from dreamsboard.engine.task_engine_builder.core import TaskEngineBuilder
from dreamsboard.engine.storage.task_step_store.simple_task_step_store import SimpleTaskStepStore
from langchain_community.chat_models import ChatOpenAI
from dreamsboard.dreams.builder_task_step.base import StructuredTaskStepStoryboard
from dreamsboard.engine.utils import concat_dirs
from dreamsboard.engine.storage.task_step_store.types import DEFAULT_PERSIST_FNAME
from dreamsboard.common.try_parse_json_object import try_parse_json_object
from dreamsboard.engine.memory.mctsr.prompt import RefineResponse
from dreamsboard.dreams.task_step_md.base import TaskStepMD
from dreamsboard.common import _get_assistants_tool

import logging
import threading
import os
from dreamsboard.dreams.task_step_to_question_chain.weaviate.prepare_load import get_query_hash
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# 控制台打印
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

logger.addHandler(handler)

In [4]:

llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
kor_dreams_task_step_llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.95,
    top_p=0.70,
)

deepseek_llm = ChatOpenAI(
    openai_api_base=os.environ.get("DEEPSEEK_API_BASE"),
    model=os.environ.get("DEEPSEEK_API_MODEL"),
    openai_api_key=os.environ.get("DEEPSEEK_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
zhipuai_llm = ChatOpenAI(
    openai_api_base=os.environ.get("ZHIPUAI_API_BASE"),
    model=os.environ.get("ZHIPUAI_API_MODEL"),
    openai_api_key=os.environ.get("ZHIPUAI_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
if 'glm' in os.environ.get("API_MODEL"):

    tools= [ { "type": "web_search",   "web_search": {"enable": False ,"search_result": False   }}]
else:
    tools = []
llm_with_tools = llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )
kor_dreams_task_step_llm_with_tools = kor_dreams_task_step_llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )

from tests.test_builder_task_step.prompts import (
    AEMO_REPRESENTATION_PROMPT_TEMPLATE as AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST,
    STORY_BOARD_SCENE_TEMPLATE as STORY_BOARD_SCENE_TEMPLATE_TEST,
    STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE as STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST,
    EDREAMS_EVOLUTIONARY_TEMPLATE as EDREAMS_EVOLUTIONARY_TEMPLATE_TEST,
    EDREAMS_PERSONALITY_TEMPLATE as EDREAMS_PERSONALITY_TEMPLATE_TEST,
    DREAMS_GEN_TEMPLATE as DREAMS_GEN_TEMPLATE_TEST,
) 
os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST



# 存储
cross_encoder_path = "/mnt/ceph/develop/jiawei/model_checkpoint/jina-reranker-v2-base-multilingual"
embed_model_path = "/mnt/ceph/develop/jiawei/model_checkpoint/m3e-base"
start_task_context = "模型蒸馏的是什么？"
builder = StructuredTaskStepStoryboard.form_builder(
    llm_runable=llm_with_tools,
    kor_dreams_task_step_llm=kor_dreams_task_step_llm_with_tools,
    start_task_context=start_task_context,
    cross_encoder_path=cross_encoder_path,
    embed_model_path=embed_model_path
)


/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
    

In [5]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=True)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
8d5fe71e-8955-44c7-b28f-c3dc8a687d90
4b640109-4354-4389-8737-291b0be13b95
9f481a8f-87fa-424b-b63f-621c3782019a
a696964e-3ceb-4b7a-915b-e78ba5a57923
d4812465-a22f-4cff-bdc6-122697d504d5
fee28969-32a9-4731-b50d-896fd4bbc166
fe0e78d6-b6a5-4ab2-b9ef-9292f062b207


In [5]:

from dreamsboard.document_loaders.structured_storyboard_loader import StructuredStoryboard
store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./44dcb3d40672df7b7ffb432745114079/storage")
 
structured_storyboard = StructuredStoryboard(json_data=[step.__dict__ for step in list(store_load.task_step_all.values())])

structured_storyboard.parse_table()

,task_step_id,shot_number,scene_number,start_task_context,aemo_representation_context,task_step_name,task_step_description,task_step_level,task_step_question,task_step_question_context,task_step_question_answer,ref_task_step_id
0,8d5fe71e-8955-44c7-b28f-c3dc8a687d90,1,story_board0,模型蒸馏的是什么？,### 分析近几年研究领域的技术框架与方法论\n在模型蒸馏相关研究领域，技术框架与方法论围绕...,分析近几年研究领域的技术框架与方法论,在模型蒸馏相关研究领域，技术框架与方法论围绕着如何将知识从一个复杂的教师模型转移到一个简单的...,0,,[],随着研究的持续推进，新的技术框架和方法论不断涌现。一些研究开始探索将强化学习的思想融入模型蒸...,
1,4b640109-4354-4389-8737-291b0be13b95,2,story_board1,模型蒸馏的是什么？,### 分析近几年研究领域的技术框架与方法论\n在模型蒸馏相关研究领域，技术框架与方法论围绕...,研究论文中采用的主要框架在不同任务中的应用与变体,在模型蒸馏中，Transformer、GAN等框架有不同应用。例如在图像任务中，基于Tran...,1,,[],在模型蒸馏的研究中，Transformer和GAN的结合已经在多个任务中展现出潜力。例如，在...,
2,9f481a8f-87fa-424b-b63f-621c3782019a,3,story_board2,模型蒸馏的是什么？,### 分析近几年研究领域的技术框架与方法论\n在模型蒸馏相关研究领域，技术框架与方法论围绕...,评估学术界的技术进步与局限性,技术进步：模型蒸馏技术在提升学生模型性能方面取得显著进展，能有效压缩模型规模同时保持较高准确...,2,,[],在对抗性样本防御方面，进一步的研究可聚焦于结合多种防御策略，如将模型蒸馏与对抗训练、鲁棒优化...,
3,a696964e-3ceb-4b7a-915b-e78ba5a57923,4,story_board3,模型蒸馏的是什么？,### 分析近几年研究领域的技术框架与方法论\n在模型蒸馏相关研究领域，技术框架与方法论围绕...,探讨计算模型在不同数据集与应用场景下的适用性与泛化能力,在不同数据集上，模型蒸馏表现各异。在大规模、多样化数据集上，模型蒸馏能更好学习通用特征，泛化...,3,,[],在具体数据集上的表现方面，模型蒸馏在ImageNet等大规模数据集上表现出色，通过知识转移机...,
4,d4812465-a22f-4cff-bdc6-122697d504d5,5,story_board4,模型蒸馏的是什么？,### 分析近几年研究领域的技术框架与方法论\n在模型蒸馏相关研究领域，技术框架与方法论围绕...,分析最新算法的稳定性与容错性,部分最新算法针对稳定性进行优化，在复杂动态环境下，通过改进蒸馏过程中的参数更新策略等方式，提...,4,,[],随着对模型蒸馏研究的深入，未来需要探索更多创新方法来提升算法的稳定性和容错性。例如，研究如何...,
5,fee28969-32a9-4731-b50d-896fd4bbc166,6,story_board5,模型蒸馏的是什么？,### 分析近几年研究领域的技术框架与方法论\n在模型蒸馏相关研究领域，技术框架与方法论围绕...,评估论文中提出的未来研究方向与挑战,未来研究方向：提出新研究问题如如何更高效地蒸馏模型中的隐式知识，改进现有蒸馏方法以适应更复杂...,5,,[],未来研究方向主要集中在如何更高效地蒸馏模型中的隐式知识。例如，研究如何量化隐式知识，并设计相...,
6,fe0e78d6-b6a5-4ab2-b9ef-9292f062b207,7,story_board6,模型蒸馏的是什么？,### 分析近几年研究领域的技术框架与方法论\n在模型蒸馏相关研究领域，技术框架与方法论围绕...,模型蒸馏的定义总结,模型蒸馏是一种将复杂教师模型中的知识转移到简单学生模型的技术，旨在在压缩模型规模的同时保持或...,6,,[],在医疗影像分析领域，模型蒸馏可助力将大型、复杂的医学图像诊断模型知识传递给小型模型，降低对硬...,




#### 场景加载模块设计方案
```

编写符合计算机科学领域的 故事情境提示词，生成研究情境（story_scenario_context），替换现有的langchain会话模板，
1、对这个提示词所要求的输入拆分成子任务，
2、对每个子任务指令转换为子问题，召回问题前3条，
3、对召回内容与问题拼接，合成会话内容变量（scene_content）


对每个子问题相关的召回内容，转换为第一人称的会话总结（研究场景（scene_monologue_context）），

1、对召回内容与问题拼接，对每个前3条组成一个总结任务的提示词，为每个任务标记唯一编号，组成任务上下文（story_board_summary_context）
2、加载编号和story_board_summary_context，转换会话信息
```



#### MCTS设计方案

```
MCT 自优化算法代表了蒙特卡洛树搜索（MCTS）与大型语言模型的结合，将不同的场景执行任务过程抽象为搜索树结构。树上的节点代表当前不同视角下的选择策略，而边表示主体对自身的反思。该算法的操作流程遵循 MCTS 算法的一般模式。

具体来说，我们采用模型的先验知识，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


1、对问题生成的子任务，生成一个合理的规划的节点
2、对每个节点创建一个MCTS任务，
3、输入 problem（总问题的子任务相关的子问题）
4、评分代码重构，将片段摘录器模块集成到一个关于_evaluate_answer逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于evaluate_system_prompt 的生成策略，具体的为编写一个关于带有评估的评估器，由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message

5、自我反思代码重构,将片段摘录器模块集成到一个关于self_refine逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于critic_system_prompt和refine_system_prompt的生成策略，critic_system_prompt为生成一个关于子问题相关的loader_cosplay_builder中自身不完美的评价内容，refine_system_prompt为不完美评价的思考过程和评分值。
具体的为编写一个关于带有评价的生成器和反思生成器，它们由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message
```



#### MCTS执行中的重要环节
```
MCTS中的约束规则如下，需要保证这些节点必须符合下面所定义的基本规则

提示约束：模型在奖励评分期间必须遵守最严格的标准。生成结果需要为JSON Response format
{
    "thought": "The thought process behind the answer.",
    "answer": "A float representing the answer to the problem."
}


高分抑制：评分节点中不存在满分反馈机制；任何超过 95 分的奖励都会按固定金额减少，以遏制过高分数。

重复采样：每次访问搜索树节点都涉及对节点奖励的重复采样，以增强自我评估的可靠性。需要注意的是，当对节点的子节点进行奖励采样时，我们也会对其父节点进行奖励采样，以增加奖励采样的样本量。
```



In [6]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

8d5fe71e-8955-44c7-b28f-c3dc8a687d90
4b640109-4354-4389-8737-291b0be13b95
9f481a8f-87fa-424b-b63f-621c3782019a
a696964e-3ceb-4b7a-915b-e78ba5a57923
d4812465-a22f-4cff-bdc6-122697d504d5
fee28969-32a9-4731-b50d-896fd4bbc166
fe0e78d6-b6a5-4ab2-b9ef-9292f062b207


In [7]:
 


def worker(step: int, task_engine: TaskEngineBuilder, task_step_store: BaseTaskStepStore):
    owner = f"step:{step}, task_step_id:{task_engine.task_step_id}, thread {threading.get_native_id()}"
    logger.info(f"{owner}，任务开始")
    if not task_engine.check_engine_init():
        task_engine.init_task_engine()
        task_engine.init_task_engine_dreams()
        task_engine.init_task_engine_storyboard_executor()

    try:
        logger.info(f"{owner}，storyboard_code_gen_builder")
        code_gen_builder = task_engine.storyboard_code_gen_builder()
        task_step = task_engine.task_step_store.get_task_step(task_engine.task_step_id)
        if task_step.task_step_question_answer is None or len(task_step.task_step_question_answer) == 0:
            task_engine.generate_step_answer(code_gen_builder)
        
        logger.info(f"step:{step}, {owner}，get_mcts_node")
        mcts_node = task_engine.get_mcts_node()
        if step % 2 == 0:
            mcts_node.llm_runable = deepseek_llm
        if step % 3 == 0:
            mcts_node.llm_runable = zhipuai_llm
        logger.info(f"step:{step}, {owner}，get_mcts_node run")
        answer = mcts_node.run()
        
        mcts_node.print()
        print(answer)
        task_step.task_step_question_answer = answer 
        task_step_id = task_engine.task_step_id
        
        task_engine.task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage/{task_step_id}", basename=DEFAULT_PERSIST_FNAME)
        task_engine.task_step_store.persist(persist_path=task_step_store_path) 
        
        task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage", basename=DEFAULT_PERSIST_FNAME)
        task_step_store.persist(persist_path=task_step_store_path) 

    except Exception as e:
        logger.error("场景加载失败", e)

    logger.info(f"{owner}，任务结束")




if __name__ == "__main__":
    threads = []
    
    step =0
    task_step_store = builder.task_step_store
    # 初始化任务引擎
    task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
    while not task_engine_builder.empty():
       
        task_engine = task_engine_builder.get()
        step += 1
        t = threading.Thread(target=worker,
                             kwargs={"step": step, "task_engine": task_engine, "task_step_store": task_step_store},
                             daemon=True)
        t.start()
        threads.append(t) 
    
    for t in threads:
        t.join()

step:1, task_step_id:8d5fe71e-8955-44c7-b28f-c3dc8a687d90, thread 281410，任务开始
step:2, task_step_id:4b640109-4354-4389-8737-291b0be13b95, thread 281411，任务开始
step:3, task_step_id:9f481a8f-87fa-424b-b63f-621c3782019a, thread 281412，任务开始
step:4, task_step_id:a696964e-3ceb-4b7a-915b-e78ba5a57923, thread 281413，任务开始
step:5, task_step_id:d4812465-a22f-4cff-bdc6-122697d504d5, thread 281414，任务开始
step:6, task_step_id:fee28969-32a9-4731-b50d-896fd4bbc166, thread 281415，任务开始
step:7, task_step_id:fe0e78d6-b6a5-4ab2-b9ef-9292f062b207, thread 281416，任务开始
step:2, task_step_id:4b640109-4354-4389-8737-291b0be13b95, thread 281411，storyboard_code_gen_builder
step:4, task_step_id:a696964e-3ceb-4b7a-915b-e78ba5a57923, thread 281413，storyboard_code_gen_builder
step:1, task_step_id:8d5fe71e-8955-44c7-b28f-c3dc8a687d90, thread 281410，storyboard_code_gen_builder
step:3, task_step_id:9f481a8f-87fa-424b-b63f-621c3782019a, thread 281412，storyboard_code_gen_builder
step:5, task_step_id:d4812465-a22f-4cff-bdc6-12269

Received: content='30' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 232, 'total_tokens': 234, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'ep-20241224194233-9mshp', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-079f8ca5-64a6-42f9-8b51-aa6679acd937-0'
Iteration complete for resource_evaluate_d4812465-a22f-4cff-bdc6-122697d504d5.
Received: content='70' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 238, 'total_tokens': 240, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'ep-20241224194233-9mshp', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6b432391-af11-4e10-b61d-76827636b089-0'
Iteration complete for resource_evaluate_8d5fe71e-8955-44c7-b28f-c3dc8a687d90.


owner:getlock thread 281491, resource_id:resource_critic_d4812465-a22f-4cff-bdc6-122697d504d5
owner:lock thread 281491, resource_id:resource_critic_d4812465-a22f-4cff-bdc6-122697d504d5
owner:event end thread 281410, _evaluate_answer
owner:register_event thread 281410
owner:getlock thread 281497, resource_id:resource_critic_8d5fe71e-8955-44c7-b28f-c3dc8a687d90
owner:lock thread 281497, resource_id:resource_critic_8d5fe71e-8955-44c7-b28f-c3dc8a687d90
resource_evaluate_9f481a8f-87fa-424b-b63f-621c3782019a: content='78' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 288, 'total_tokens': 291, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-59eed43d-fe60-4fb2-a4c9-aab7ab81124a-0'


Received: content='78' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 288, 'total_tokens': 291, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-59eed43d-fe60-4fb2-a4c9-aab7ab81124a-0'
Iteration complete for resource_evaluate_9f481a8f-87fa-424b-b63f-621c3782019a.


resource_evaluate_fe0e78d6-b6a5-4ab2-b9ef-9292f062b207: content='30' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 237, 'total_tokens': 239, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'ep-20241224194233-9mshp', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3ca2950a-bca3-4923-be3b-d72c4b2238cc-0'
resource_evaluate_a696964e-3ceb-4b7a-915b-e78ba5a57923: content='85' response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 445, 'total_tokens': 446, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 384}, 'prompt_cache_hit_tokens': 384, 'prompt_cache_miss_tokens': 61}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3a5770e1b4', 'finish_reason': 'stop', 'logprobs': None} id='run-13b207c0-0877-4150-b0a5-40a1c67e8740-0'
owner:event end thread 281412, _evaluate_answer
owner:registe

...

在具体实现中，Smith的方法通过构建特征依赖图，量化了隐式知识在网络层间的传递路径。具体而言，该方法首先提取教师模型各层的特征表示，然后构建一个图结构来表示这些特征之间的依赖关系。通过图神经网络对这些依赖关系进行建模，实现了隐式知识的量化。在图像分类任务中，该方法显著提升了学生模型的分类精度，相比传统蒸馏方法提高了5%。与其他隐式知识蒸馏方法相比，Smith的方法在捕捉细粒度特征方面表现出色，但在计算复杂度上较高。

Johnson的框架通过任务特定的注意力机制，动态调整不同任务间的知识转移权重。具体来说，该机制首先计算每个任务的重要性分数，然后根据这些分数动态调整从教师模型到学生模型的知识转移权重。在自然语言处理的多任务学习中，该方法使得模型在各项任务上的平均性能提升了10%。例如，在情感分析和文本分类的联合任务中，模型的表现显著优于单一任务模型。

计算复杂度高的问题主要源于特征依赖图的构建和图神经网络的计算需求。可能的解决方案包括优化图结构、采用高效的图神经网络算法等。参数调整敏感性问题则表现为模型性能对超参数的选择较为敏感，可以通过自动化调参技术如贝叶斯优化来缓解这一问题。

在医疗影像分析领域，模型蒸馏技术结合医学领域的先验知识，如在COVID-19影像分析中，通过引入肺部结构的先验知识，提升了模型对病变区域的识别准确率。在自动驾驶领域，通过融合摄像头、雷达等多传感器数据，模型蒸馏技术实现了更高效的决策模型，显著提升了车辆的行驶安全性。

基于对抗训练的鲁棒性蒸馏框架通过生成对抗性样本，模拟复杂环境下的数据分布变化。具体来说，该方法首先生成与真实数据分布相似的对抗性样本，然后在这些样本上训练学生模型，从而提升其鲁棒性。然而，对抗性样本的生成过程计算成本较高，可以通过简化生成网络或采用高效的对抗样本生成算法来降低成本。为避免过拟合，可以引入正则化项或采用数据增强策略。

在未来的研究中，探索更高效的知识蒸馏方法，如自监督学习和元学习的结合，将是提升模型在复杂任务中表现的重要方向。此外，设计更复杂的知识融合机制，如基于图神经网络的知识蒸馏，也是未来研究的重要方向。通过这些方法，模型蒸馏在多模态和复杂任务中的表现有望得到进一步提升。

在跨领域应用中，如何有效融合不同领域的先验知识，以及如何在保持模型性能的同时降低计算成本，将是需要进一步探索的挑战。通过不断优

resource_refine_8d5fe71e-8955-44c7-b28f-c3dc8a687d90: content='### 优化后的回答: \n在模型蒸馏领域，多种先进技术框架被广泛应用于不同任务以实现知识从教师模型到学生模型的转移。例如，Transformer框架在图像任务中发挥着重要作用。其自注意力机制能够有效捕捉图像中的长距离依赖关系，教师模型利用这一特性学习到丰富的图像特征知识。在模型蒸馏时，通过注意力图等方式将这些特征知识传递给学生模型，帮助学生模型更好地理解图像内容，从而实现知识转移。在生成任务中，GAN框架有所应用。生成器生成的数据可以作为额外的信息源，教师模型基于这些生成数据学习到更广泛的知识，然后通过蒸馏将这些知识传授给学生模型，使得学生模型在生成任务中能够生成更优质的内容。\n\n从技术进步方面来看，模型蒸馏在提升学生模型性能上取得了显著成果。通过知识转移，学生模型能够在不具备大规模计算资源的情况下，达到接近教师模型的性能表现。同时，在压缩模型规模的同时保持较高的准确率，使得模型能够在资源受限的设备上高效运行。然而，模型蒸馏也存在一些局限性。在复杂任务中，学生模型往往难以完全复现教师模型的知识，存在模型偏差问题。并且，高质量的标注数据对蒸馏效果影响较大，数据依赖问题较为突出。\n\n在数据集适用性方面，模型蒸馏在大规模、多样化数据集上通常表现良好，能够充分利用数据中的丰富信息实现有效的知识转移。但在小众、特定领域数据集上，由于数据量有限且特征较为独特，模型蒸馏可能面临挑战。在多领域、多模态数据场景下，不同领域和模态的数据特点差异大，如何将教师模型在不同数据上学习到的知识准确地转移到学生模型是一大难题。解决这些问题需要围绕知识转移机制，例如针对特定领域数据设计更适配的知识转移方式，对于多模态数据采用融合不同模态知识的转移策略。\n\n在算法稳定性与容错性方面，部分最新算法针对稳定性进行了优化。比如改进蒸馏过程中的参数更新策略，通过更合理地调整参数更新步长，使得模型在蒸馏过程中更加稳定。然而，在复杂环境下，模型蒸馏的容错性仍有待提高，例如在数据存在噪声或模型结构复杂时，算法可能容易出现不稳定的情况。\n\n未来研究方向包括如何更高效地蒸馏模型中的隐式知识。教师模型中存在大量难以直接提取的隐式知识，若能有效将这些知识转移到学生模型，将进一步提

Received: content='### 优化后的回答: \n在模型蒸馏领域，多种先进技术框架被广泛应用于不同任务以实现知识从教师模型到学生模型的转移。例如，Transformer框架在图像任务中发挥着重要作用。其自注意力机制能够有效捕捉图像中的长距离依赖关系，教师模型利用这一特性学习到丰富的图像特征知识。在模型蒸馏时，通过注意力图等方式将这些特征知识传递给学生模型，帮助学生模型更好地理解图像内容，从而实现知识转移。在生成任务中，GAN框架有所应用。生成器生成的数据可以作为额外的信息源，教师模型基于这些生成数据学习到更广泛的知识，然后通过蒸馏将这些知识传授给学生模型，使得学生模型在生成任务中能够生成更优质的内容。\n\n从技术进步方面来看，模型蒸馏在提升学生模型性能上取得了显著成果。通过知识转移，学生模型能够在不具备大规模计算资源的情况下，达到接近教师模型的性能表现。同时，在压缩模型规模的同时保持较高的准确率，使得模型能够在资源受限的设备上高效运行。然而，模型蒸馏也存在一些局限性。在复杂任务中，学生模型往往难以完全复现教师模型的知识，存在模型偏差问题。并且，高质量的标注数据对蒸馏效果影响较大，数据依赖问题较为突出。\n\n在数据集适用性方面，模型蒸馏在大规模、多样化数据集上通常表现良好，能够充分利用数据中的丰富信息实现有效的知识转移。但在小众、特定领域数据集上，由于数据量有限且特征较为独特，模型蒸馏可能面临挑战。在多领域、多模态数据场景下，不同领域和模态的数据特点差异大，如何将教师模型在不同数据上学习到的知识准确地转移到学生模型是一大难题。解决这些问题需要围绕知识转移机制，例如针对特定领域数据设计更适配的知识转移方式，对于多模态数据采用融合不同模态知识的转移策略。\n\n在算法稳定性与容错性方面，部分最新算法针对稳定性进行了优化。比如改进蒸馏过程中的参数更新策略，通过更合理地调整参数更新步长，使得模型在蒸馏过程中更加稳定。然而，在复杂环境下，模型蒸馏的容错性仍有待提高，例如在数据存在噪声或模型结构复杂时，算法可能容易出现不稳定的情况。\n\n未来研究方向包括如何更高效地蒸馏模型中的隐式知识。教师模型中存在大量难以直接提取的隐式知识，若能有效将这些知识转移到学生模型，将进一步提升学生模型的性能。同时，需要改进现有蒸馏方法以适应更复杂的任务，例如设计更灵活的知识转移

解析后的 JSON 对象: {'thought': '在模型蒸馏领域，多种先进技术框架被广泛应用于不同任务以实现知识从教师模型到学生模型的转移。例如，Transformer框架在图像任务中发挥着重要作用。其自注意力机制能够有效捕捉图像中的长距离依赖关系，教师模型利用这一特性学习到丰富的图像特征知识。在模型蒸馏时，通过注意力图等方式将这些特征知识传递给学生模型，帮助学生模型更好地理解图像内容，从而实现知识转移。在生成任务中，GAN框架有所应用。生成器生成的数据可以作为额外的信息源，教师模型基于这些生成数据学习到更广泛的知识，然后通过蒸馏将这些知识传授给学生模型，使得学生模型在生成任务中能够生成更优质的内容。\n\n从技术进步方面来看，模型蒸馏在提升学生模型性能上取得了显著成果。通过知识转移，学生模型能够在不具备大规模计算资源的情况下，达到接近教师模型的性能表现。同时，在压缩模型规模的同时保持较高的准确率，使得模型能够在资源受限的设备上高效运行。然而，模型蒸馏也存在一些局限性。在复杂任务中，学生模型往往难以完全复现教师模型的知识，存在模型偏差问题。并且，高质量的标注数据对蒸馏效果影响较大，数据依赖问题较为突出。\n\n在数据集适用性方面，模型蒸馏在大规模、多样化数据集上通常表现良好，能够充分利用数据中的丰富信息实现有效的知识转移。但在小众、特定领域数据集上，由于数据量有限且特征较为独特，模型蒸馏可能面临挑战。在多领域、多模态数据场景下，不同领域和模态的数据特点差异大，如何将教师模型在不同数据上学习到的知识准确地转移到学生模型是一大难题。解决这些问题需要围绕知识转移机制，例如针对特定领域数据设计更适配的知识转移方式，对于多模态数据采用融合不同模态知识的转移策略。\n\n在算法稳定性与容错性方面，部分最新算法针对稳定性进行了优化。比如改进蒸馏过程中的参数更新策略，通过更合理地调整参数更新步长，使得模型在蒸馏过程中更加稳定。然而，在复杂环境下，模型蒸馏的容错性仍有待提高，例如在数据存在噪声或模型结构复杂时，算法可能容易出现不稳定的情况。\n\n未来研究方向包括如何更高效地蒸馏模型中的隐式知识。教师模型中存在大量难以直接提取的隐式知识，若能有效将这些知识转移到学生模型，将进一步提升学生模型的性能。同时，需要改进现有蒸馏方法以适应更复杂的任务，例如设计更灵活的知识转移架构，使学生

此外，研究人员还在探索如何利用元学习的思想来优化模型蒸馏过程。元学习旨在让模型快速适应新任务，在模型蒸馏中引入元学习，可以使学生模型更快地学习到教师模型的知识模式。例如，通过元学习算法学习到不同任务下知识转移的通用策略，然后将这些策略应用到具体的模型蒸馏任务中，帮助学生模型更高效地从教师模型获取知识。  


In [8]:

store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./44dcb3d40672df7b7ffb432745114079/storage")
task_step_md = TaskStepMD(store_load)
md_text =   task_step_md.format_md() 
from IPython.display import Markdown, display

display(Markdown(md_text.text))


# 模型蒸馏的是什么？ 


### 分析近几年研究领域的技术框架与方法论 [task_id:0](8d5fe71e-8955-44c7-b28f-c3dc8a687d90)

此外，研究人员还在探索如何利用元学习的思想来优化模型蒸馏过程。元学习旨在让模型快速适应新任务，在模型蒸馏中引入元学习，可以使学生模型更快地学习到教师模型的知识模式。例如，通过元学习算法学习到不同任务下知识转移的通用策略，然后将这些策略应用到具体的模型蒸馏任务中，帮助学生模型更高效地从教师模型获取知识。

### 研究论文中采用的主要框架在不同任务中的应用与变体 [task_id:1](4b640109-4354-4389-8737-291b0be13b95)

在模型蒸馏的研究中，Transformer和GAN的结合已经在多个任务中展现出潜力。例如，在图像生成任务中，研究者们通过将Transformer的自注意力机制与GAN的生成能力相结合，成功生成了高质量的图像。具体来说，Transformer的自注意力机制能够捕捉图像中的全局依赖关系，而GAN则能够生成逼真的图像细节。这种结合不仅提升了图像生成的质量，还为多模态学习提供了新的思路。此外，改进损失函数设计也是提升模型性能的关键。例如，对比学习损失通过最大化正样本对的相似性和最小化负样本对的相似性，增强了模型的判别能力。自适应损失则根据任务需求动态调整损失权重，提高了模型的泛化能力。在多模态学习中，跨模态的注意力机制和联合训练策略能够有效融合不同模态的知识，从而提升模型的整体性能。在跨领域迁移学习中，模型蒸馏技术的应用也取得了显著成果。例如，在医学图像分析领域，通过迁移学习从自然图像中提取的知识，显著提升了医学图像分类的准确性。这些研究成果不仅推动了模型蒸馏技术的发展，还为解决实际应用中的复杂问题提供了新的思路。

### 评估学术界的技术进步与局限性 [task_id:2](9f481a8f-87fa-424b-b63f-621c3782019a)

然而，模型蒸馏技术仍存在一定的局限性。首先，模型偏差问题在不同任务中表现各异。在图像识别任务中，学生模型在细粒度分类上表现出较大的偏差，准确率下降约10%。在自然语言处理任务中，长文本处理能力显著减弱，导致性能下降约15%。其次，数据依赖问题尤为突出，高质量标注数据能提升蒸馏效果20%以上，而低质量数据则可能导致性能下降15%。建议采用数据增强和半监督学习技术来缓解这一问题。此外，模型蒸馏在跨领域应用中也面临挑战，如在医疗图像分析中，从自然图像中提取的知识可能不完全适用于医学图像，导致分类准确率仅提升5%。未来研究可聚焦于改进知识转移机制，探索新的模型结构，以进一步提升模型蒸馏的效果和泛化能力。同时，还需关注模型蒸馏在不同领域的实际应用，如自然语言处理、推荐系统等，分析其在具体行业（如医疗、金融）中的实际应用案例，为解决复杂问题提供新的思路。

### 探讨计算模型在不同数据集与应用场景下的适用性与泛化能力 [task_id:3](a696964e-3ceb-4b7a-915b-e78ba5a57923)

在数据稀缺性和标注质量方面，基于数据增强和迁移学习的策略在小众数据集上的应用也取得了显著进展。例如，在医学影像数据集中，通过引入外部数据源和领域特定的数据增强技术，模型能够从有限的样本中学习到更多的特征。具体来说，数据增强技术如随机旋转、缩放和翻转能够有效增加训练数据的多样性，而迁移学习则通过从相关领域（如自然图像）中迁移知识，提升了模型在小众数据集上的表现。实验结果表明，基于数据增强和迁移学习的策略在医学影像数据集上的分类准确率提升了15%。

### 分析最新算法的稳定性与容错性 [task_id:4](d4812465-a22f-4cff-bdc6-122697d504d5)

此外，量化技术作为模型压缩的一种重要手段，也能与模型蒸馏相结合来提升算法性能。量化是将模型中的参数和激活值用低比特表示，从而减少存储和计算成本。在自然语言处理的情感分析任务中，对BERT模型进行量化处理，将32位浮点数参数量化为8位整数。量化过程会不可避免地带来一定信息损失，影响模型性能。此时引入模型蒸馏，以未量化的教师模型指导量化后的学生模型学习。通过蒸馏，量化后的模型在情感分类准确率上比单纯量化的模型提高了约5%，并且在处理不同风格文本数据时，表现出更好的稳定性和容错性。这是因为模型蒸馏帮助量化后的模型弥补了量化过程中丢失的信息，使得模型在面对不同文本风格导致的数据分布差异时，依然能够准确提取关键语义信息进行情感判断。

### 评估论文中提出的未来研究方向与挑战 [task_id:5](fee28969-32a9-4731-b50d-896fd4bbc166)

在具体实现中，Smith的方法通过构建特征依赖图，量化了隐式知识在网络层间的传递路径。具体而言，该方法首先提取教师模型各层的特征表示，然后构建一个图结构来表示这些特征之间的依赖关系。通过图神经网络对这些依赖关系进行建模，实现了隐式知识的量化。在图像分类任务中，该方法显著提升了学生模型的分类精度，相比传统蒸馏方法提高了5%。与其他隐式知识蒸馏方法相比，Smith的方法在捕捉细粒度特征方面表现出色，但在计算复杂度上较高。

Johnson的框架通过任务特定的注意力机制，动态调整不同任务间的知识转移权重。具体来说，该机制首先计算每个任务的重要性分数，然后根据这些分数动态调整从教师模型到学生模型的知识转移权重。在自然语言处理的多任务学习中，该方法使得模型在各项任务上的平均性能提升了10%。例如，在情感分析和文本分类的联合任务中，模型的表现显著优于单一任务模型。

计算复杂度高的问题主要源于特征依赖图的构建和图神经网络的计算需求。可能的解决方案包括优化图结构、采用高效的图神经网络算法等。参数调整敏感性问题则表现为模型性能对超参数的选择较为敏感，可以通过自动化调参技术如贝叶斯优化来缓解这一问题。

在医疗影像分析领域，模型蒸馏技术结合医学领域的先验知识，如在COVID-19影像分析中，通过引入肺部结构的先验知识，提升了模型对病变区域的识别准确率。在自动驾驶领域，通过融合摄像头、雷达等多传感器数据，模型蒸馏技术实现了更高效的决策模型，显著提升了车辆的行驶安全性。

基于对抗训练的鲁棒性蒸馏框架通过生成对抗性样本，模拟复杂环境下的数据分布变化。具体来说，该方法首先生成与真实数据分布相似的对抗性样本，然后在这些样本上训练学生模型，从而提升其鲁棒性。然而，对抗性样本的生成过程计算成本较高，可以通过简化生成网络或采用高效的对抗样本生成算法来降低成本。为避免过拟合，可以引入正则化项或采用数据增强策略。

在未来的研究中，探索更高效的知识蒸馏方法，如自监督学习和元学习的结合，将是提升模型在复杂任务中表现的重要方向。此外，设计更复杂的知识融合机制，如基于图神经网络的知识蒸馏，也是未来研究的重要方向。通过这些方法，模型蒸馏在多模态和复杂任务中的表现有望得到进一步提升。

在跨领域应用中，如何有效融合不同领域的先验知识，以及如何在保持模型性能的同时降低计算成本，将是需要进一步探索的挑战。通过不断优化算法和探索新的应用场景，模型蒸馏技术有望在更多领域发挥重要作用。

### 模型蒸馏的定义总结 [task_id:6](fe0e78d6-b6a5-4ab2-b9ef-9292f062b207)

在教育领域，模型蒸馏也有着潜在的应用价值。例如，将大型的教育知识图谱模型的知识蒸馏到小型模型中，能够为学生提供更加个性化且高效的学习辅助工具。小型模型可以在移动设备上运行，随时随地为学生解答问题、提供学习建议。在金融领域，模型蒸馏可助力将复杂的风险评估模型知识传递给轻量级模型，使得在资源有限的情况下，依然能够快速准确地进行风险评估，为投资决策提供有力支持。


 



StructuredTaskStepStoryboard
    
对任务进行规划，生成段落之间组成一个动态上下文，  扩写任务步骤构建MCTS任务
    输入：
    	start_task_context： 起始任务
    	llm： 模型
    	kor_dreams_task_step_llm: 任务抽取模型
    	task_step_store: 任务存储器（SimpleTaskStepStore）
    	cross_encoder_path: ranking模型路径（sentence_transformers），当前设计耦合了业务，之后会单独设计一个召回模块

    任务：

        1、对任务（AEMO_REPRESENTATION_PROMPT_TEMPLATE）按照提示词要求进行扩写，将扩写任务步骤收集 （src/dreamsboard/dreamsboard/engine/entity/task_step、src/dreamsboard/tests/test_kor/test_kor3.py）

        2、收集每个任务后存储到磁盘（src/dreamsboard/dreamsboard/engine/storage/task_step_store）

        3、对每个子任务载入会话场景，然后按照扩写任务步骤构建，MCTS任务 loader_task_step_iter_builder


	》 TaskEngineBuilder 场景加载模块
		执行会话场景资源初始化，构建MCTS任务

    根据任务步骤，构建场景加载模块，生成资源文件csv
    根据每个任务，载入StructuredDreamsStoryboard 会话场景
    按照扩写任务步骤构建MCTS任务

		输入：
			task_step_id
			task_step_store: 任务存储器（SimpleTaskStepStore）
			start_task_context： 起始任务
			llm： 模型

		任务：
			init_task_engine：
				初始化任务引擎
        》TaskStepToQuestionChain 
        	输入：
        		client： 矢量库客户端
        		llm： 模型
					invoke_task_step_to_question：1、 对开始任务进行抽取，得到任务步骤，提示词所要求的输入拆分成子任务， 
					invoke_task_step_question_context： 2、对每个子任务指令转换为子问题，召回问题前3条，对任务步骤进行抽取，得到任务步骤的上下文
					export_csv_file_path: 3、对召回内容与问题 导出csv文件

			init_task_engine_dreams
				初始化场景加载资源
					StoryBoardDreamsGenerationChain
					对每个子任务通过职业提示词，载入会话场景
						1、构建场景信息（story_scenario_context），提示词（STORY_BOARD_SCENE_TEMPLATE）
						2、对任务上下文(story_board_summary_context)，构建第一人称数据(scene_monologue_context),提示词（STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE）
						3、对任务上下文(story_board_summary_context)，获取任务分析(evolutionary_step), 提示词（EDREAMS_EVOLUTIONARY_TEMPLATE）
						4、对任务分析(evolutionary_step)，分析对话预设信息（性格）， 提示词（EDREAMS_PERSONALITY_TEMPLATE）
						5、对任务上下文(story_board_summary_context)，场景信息story_scenario_context, 第一人称数据(scene_monologue_context)，
						生成关于人物职业的引导话术，提示词（DREAMS_GEN_TEMPLATE）

			init_task_engine_storyboard_executor

				构建会话场景执行器 StructuredDreamsStoryboard
					对剧本和分析结果进行结构化，将开放问题与性格分析结果进行结合。生成情景扮演会话场景执行器
			    此过程如下
			        对开放问题结果进行抽取，得到问题内容
			        对性格分析结果进行抽取，得到性格分析结果
			        增加系统提示词
			        根据剧本与任务性格基础情景扮演代码，根据每步的抽取析得到的问题，生成问题的答案
			       	在上下文中增加，关于人物职业的引导话术
	        		导出会话场景执行器
					
	    storyboard_code_gen_builder
	    	构建会话场景执行器

	    generate_step_answer
	    	获取主进程子任务的答案
	    
	    get_mcts_node
	    	构建MCTS树, 初始化当前任务相关的MCTS节点，并返回MCTS执行器
